# Purpose

To download the metasploit JSON file and add a column to the CVE processed JSON file indicating for which CVEs there is a known metasploit exploit

In [50]:
import urllib.request
import json
import re
import pandas as pd

In [69]:
FILE_LOC = 'https://raw.githubusercontent.com/rapid7/metasploit-framework/master/db/modules_metadata_base.json'
METASPLOIT_RAW = '../../data/raw/metasploit.json'
CVE_PROCESSED = '../../data/processed/cves.json'
CVE_METASPLOIT_PROCESSED = '../../data/processed/cves_metasploit.json'

In [20]:
response = urllib.request.urlopen(FILE_LOC)
raw_file =  response.read()

with open(METASPLOIT_RAW, 'wb') as fout:
    fout.write(raw_file)
print("wrote metasploit.json")

wrote metasploit.json


In [40]:
# read metasploit file
with open(METASPLOIT_RAW, 'r') as fin:
    metasploit = json.loads(fin.read())

In [51]:
# read cve file as pandas
cves_df = pd.read_json(CVE_PROCESSED)
    
# add metasploit column set to 0
cves_df['metasploit'] = 0

cves_df.head()

,access,authentication,availability,complexity,confidentiality,id,integrity,metasploit
0,NETWORK,NONE,PARTIAL,LOW,NONE,CVE-1999-0001,NONE,0
1,NETWORK,NONE,COMPLETE,LOW,COMPLETE,CVE-1999-0002,COMPLETE,0
2,NETWORK,NONE,COMPLETE,LOW,COMPLETE,CVE-1999-0003,COMPLETE,0
3,NETWORK,NONE,PARTIAL,LOW,NONE,CVE-1999-0004,NONE,0
4,NETWORK,NONE,COMPLETE,LOW,COMPLETE,CVE-1999-0005,COMPLETE,0


In [70]:
cve_regx = re.compile(r'CVE-\d{4}-\d{4,7}')
total = len(metasploit)
count = 0
for key, item in metasploit.items():
    for reference in item['references']:
        match = cve_regx.match(reference)
        if match:
            matched_cve = match.group()
            cves_df.loc[cves_df['id']==matched_cve, 'metasploit'] = 1
            
    count += 1
    print("\r{0:.2f} Complete...".format(count*100/total), end='')

100.00 Complete...

In [67]:
cves_df.head()

,access,authentication,availability,complexity,confidentiality,id,integrity,metasploit
0,NETWORK,NONE,PARTIAL,LOW,NONE,CVE-1999-0001,NONE,0
1,NETWORK,NONE,COMPLETE,LOW,COMPLETE,CVE-1999-0002,COMPLETE,0
2,NETWORK,NONE,COMPLETE,LOW,COMPLETE,CVE-1999-0003,COMPLETE,0
3,NETWORK,NONE,PARTIAL,LOW,NONE,CVE-1999-0004,NONE,0
4,NETWORK,NONE,COMPLETE,LOW,COMPLETE,CVE-1999-0005,COMPLETE,0


In [71]:
out = cves_df.to_json(orient='records')
with open(CVE_METASPLOIT_PROCESSED, 'w') as fout:
    fout.write(out)
